In [1]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain import OpenAI, SerpAPIWrapper, LLMChain

In [2]:
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    )
]

In [3]:
prefix = """Answer the following questions as best you can, but speaking as a pirate might speak. You have access to the following tools:"""
suffix = """Begin! Remember to speak as a pirate when giving your final answer. Use lots of "Args"

Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools, prefix=prefix, suffix=suffix, input_variables=["input", "agent_scratchpad"]
)

In [4]:
print(prompt.template)

Answer the following questions as best you can, but speaking as a pirate might speak. You have access to the following tools:

Search: useful for when you need to answer questions about current events

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to speak as a pirate when giving your final answer. Use lots of "Args"

Question: {input}
{agent_scratchpad}


In [5]:
llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=prompt)

In [6]:
tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)

In [7]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

In [8]:
agent_executor.run("How many people live in canada as of 2023?")



> Entering new  chain...
Thought: I need to find out the population of Canada in 2023
Action: Search
Action Input: Population of Canada in 2023
Observation: Demographic estimates for Canada, the provinces and the territories on April 1, 2023 are now available. These new data allow for an analysis of population growth during the first quarter. Canada's population was estimated at 39,858,480 on April 1, 2023, an increase of 292,232 people (+0.7%) from January 1, 2023.
Thought: I now know the final answer
Final Answer: Arrr, Canada be havin' 39,858,480 people livin' there as of 2023!

> Finished chain.


"Arrr, Canada be havin' 39,858,480 people livin' there as of 2023!"

In [9]:
prefix = """Answer the following questions as best you can. You have access to the following tools:"""
suffix = """When answering, you MUST speak in the following language: {language}.

Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "language", "agent_scratchpad"],
)

In [10]:
llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=prompt)

In [11]:
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools)

In [12]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

In [13]:
agent_executor.run(
    input="How many people live in canada as of 2023?", language="italian"
)



> Entering new  chain...
Thought: I should look for recent population estimates.
Action: Search
Action Input: Canada population 2023
Observation: 38,739,599
Thought: I should double check this number.
Action: Search
Action Input: Canada population estimates 2023
Observation: Canada's population was estimated at 39,858,480 on April 1, 2023, an increase of 292,232 people (+0.7%) from January 1, 2023.
Thought: I now know the final answer.
Final Answer: La popolazione del Canada è stata stimata a 39.858.480 il 1° aprile 2023, un aumento di 292.232 persone (+0,7%) rispetto al 1° gennaio 2023.

> Finished chain.


'La popolazione del Canada è stata stimata a 39.858.480 il 1° aprile 2023, un aumento di 292.232 persone (+0,7%) rispetto al 1° gennaio 2023.'